In [1]:
# Get data to work with

from observatorio_laboral.offer.offer_controller import OfferController
from observatorio_laboral.offer.date_range import DateRange

text_fields = []
oc = OfferController(text_fields = ["Job Title", "Description", "Qualifications"])
date_range = DateRange(1, 2013, 5, 2017)
source = "symplicity"

oc.load_offers(source, date_range)

offer_texts = oc.get_text()
offer_classes = oc.get_field_classes("Areas")

data = offer_texts
classes = offer_classes


In [2]:
# Check quantity per labels
offer_by_labels = {}
for offer in oc.offers:
    labels = " ".join(sorted(oc.get_field_classes("Areas", offer)))
    if labels not in offer_by_labels:
        offer_by_labels[labels] = []
        
    offer_by_labels[labels].append(offer)

In [3]:
qnt_list = [(label.split(), len(offers)) for label, offers in offer_by_labels.items()]
qnt_list.sort(key = lambda k: k[1], reverse=True)

In [4]:
qnt_list

[(['P'], 130),
 (['F', 'P'], 123),
 (['F'], 83),
 (['EM', 'F'], 33),
 (['EM'], 26),
 (['F', 'MC'], 20),
 (['MC'], 18),
 (['MC', 'P'], 12),
 (['F', 'MC', 'P'], 7),
 (['EM', 'MC'], 6),
 (['MC', 'TE'], 5),
 (['EM', 'F', 'MC'], 4),
 (['F', 'MC', 'OI', 'TE'], 3),
 (['EI', 'F'], 2),
 (['F', 'MC', 'TE'], 2),
 (['F', 'OI', 'P'], 2),
 (['P', 'TE'], 2),
 (['OI'], 1),
 (['OI', 'P'], 1),
 (['EM', 'F', 'MC', 'TE'], 1),
 (['MC', 'P', 'TE'], 1),
 (['F', 'OI'], 1),
 (['F', 'TE'], 1),
 (['MC', 'OI', 'TE'], 1),
 (['EI', 'F', 'P'], 1),
 (['F', 'MC', 'OI'], 1),
 (['EI'], 1),
 (['EI', 'MC'], 1),
 (['EI', 'MC', 'OI', 'TE'], 1),
 (['EM', 'OI'], 1)]

In [5]:
qnt_by_lab = {}
for labels, qnt in qnt_list:
    for lab in labels:
        if lab not in qnt_by_lab: 
            qnt_by_lab[lab] = 0
            
        qnt_by_lab[lab] += qnt
        
import operator
print(sorted(qnt_by_lab.items(), key=operator.itemgetter(1), reverse=True))


[('F', 284), ('P', 279), ('MC', 83), ('EM', 71), ('TE', 17), ('OI', 12), ('EI', 6)]


In [7]:
# Over-sampling approach
X = data
y = classes

#from imblearn.over_sampling import RandomOverSampler
#ros = RandomOverSampler(random_state=42)
#X_resampled, y_resampled = ros.fit_sample(X,y)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42)

mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)


In [8]:
# Let's see the features
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

tfidf_vect = TfidfVectorizer(ngram_range=(1,2), max_df=0.55, binary=True, norm=None, use_idf=False)

#tfidf_vect = TfidfVectorizer(ngram_range=(1,2), max_df=0.55)
tfidf_vect.fit(X)
len(tfidf_vect.vocabulary_)


44266

In [9]:
# Features by label
tfidf = tfidf_vect.transform(X)
X_inv = tfidf_vect.inverse_transform(tfidf)

features_by_label = {}
for x, labels in zip(X_inv, y):
    for label in labels:
        if label not in features_by_label:
            features_by_label[label] = {}
            
        for word in x:
            if word not in features_by_label[label]:
                features_by_label[label][word] = 0
                
            features_by_label[label][word] += 1   


In [10]:
import operator

for label in features_by_label:
    print("Label: ", label, "Words: ", len(features_by_label[label]))    
    for item in sorted(features_by_label[label].items(), key=operator.itemgetter(1), reverse=True):
        pass
        #print(item)
    #print()    

Label:  MC Words:  11072
Label:  EM Words:  8727
Label:  OI Words:  2935
Label:  F Words:  29117
Label:  P Words:  30983
Label:  EI Words:  1771
Label:  TE Words:  2350


In [11]:
# Exclusive Features by labels
exc = {}
for label, words in features_by_label.items():
    exc[label] = {}
    for word in words:
        found = False
        for l2, w2 in features_by_label.items():            
            if l2!= label:
                if word in w2:
                    found = True
        if not found:
            exc[label][word] = words[word]                

In [12]:
for label in exc:
    print("Label: ", label, "Words: ", len(exc[label]))
    for item in sorted(exc[label].items(), key=operator.itemgetter(1), reverse=True):
        print(item)
        
    print()

Label:  MC Words:  1247
('boletín', 3)
('investigación desarrollo', 3)
('investigación ejecución', 2)
('su cv', 2)
('del boletín', 2)
('de consultorías', 2)
('los cursos', 2)
('empírica', 2)
('económica social', 2)
('evidencia empírica', 2)
('de evidencia', 2)
('30 horas', 2)
('por calidad', 2)
('de literatura', 2)
('teórica de', 2)
('relacionada al', 2)
('clase', 2)
('teórica', 2)
('forma contenido', 2)
('literatura teórica', 2)
('empírica relacionada', 2)
('socioeconómicas', 2)
('analistas miembros', 2)
('prácticos', 2)
('en cursos', 1)
('investigación documentos', 1)
('auditoría mercado', 1)
('sería útil', 1)
('del trio2', 1)
('distributivas', 1)
('movimientos de', 1)
('mercado capacidad', 1)
('analista sectores', 1)
('actores canales', 1)
('básica aplicada', 1)
('mundiales elaboración', 1)
('el horario', 1)
('redactando informes', 1)
('aplicados orientados', 1)
('plan para', 1)
('informes etc', 1)
('aeropuertos', 1)
('capitales control', 1)
('jessica', 1)
('de proveedurías', 1)
('c

('por consumo', 1)
('un eficiente', 1)
('mercado regional', 1)
('contabilidad elaboración', 1)
('trabajarás', 1)
('lleva el', 1)
('interbancarias valorados', 1)
('monthly trial', 1)
('isa', 1)
('deuda agencias', 1)
('los abogados', 1)
('canje envío', 1)
('qué ofrece', 1)
('cobranza domiciliaria', 1)
('familiaridad con', 1)
('seguros deseable', 1)
('vehículos menores', 1)
('cobrar clientes', 1)
('minorista mic', 1)
('provisiones requerimiento', 1)
('crédito captación', 1)
('corporativo elaboración', 1)
('productos agrícolas', 1)
('soporte sus', 1)
('clientes apoyar', 1)
('individuales mensuales', 1)
('mismas se', 1)
('sectorial para', 1)
('de financial', 1)
('características establecidas', 1)
('empresa ya', 1)
('convertirte en', 1)
('condiciones empresariales', 1)
('sistemas electrónicos', 1)
('técnicos presentar', 1)
('esto conlleva', 1)
('recaudo definición', 1)
('asset allocation', 1)
('permitirá agregar', 1)
('financieras centrales', 1)
('sbs relacionadas', 1)
('crédito revisar', 1)

('días efectivo', 1)
('encuentren acordes', 1)
('siniestros en', 1)
('tchns elaborar', 1)
('considerará en', 1)
('mercado vales', 1)
('efectúen los', 1)
('cónyuge', 1)
('locales instalaciones', 1)
('trámite ágil', 1)
('intangibles interactuar', 1)
('forcasting finanzas', 1)
('ajustes las', 1)
('tesorería gerencia', 1)
('físicos anuales', 1)
('sido calificados', 1)
('mancomunado con', 1)
('variaciones presupuestales', 1)
('obligaciones laborales', 1)
('central apoyo', 1)
('anuales elaboración', 1)
('colocado índice', 1)
('crédito desembolsos', 1)
('profundo conocimiento', 1)
('empresas proyección', 1)
('clientes definir', 1)
('cuales realizando', 1)
('otorgados errores', 1)
('portafolio rentabilidad', 1)
('funciones señaladas', 1)
('las definiciones', 1)
('confianza contactar', 1)
('créditos velar', 1)
('significativo', 1)
('provisiones etc', 1)
('promuevan el', 1)
('sbs sobre', 1)
('hoc realizados', 1)
('otros encargados', 1)
('negociaciones diferentes', 1)
('balance mtb', 1)
('benefic

('citas con', 1)
('comerciales interdisciplinario', 1)
('detalles postular', 1)
('tdr', 1)
('sectores generar', 1)
('management compulsory', 1)
('involucren coordinación', 1)
('instituciones similares', 1)
('por rubros', 1)
('cambiantes', 1)
('puesto formular', 1)
('sociales prestados', 1)
('sourcing', 1)
('área preparación', 1)
('árbol', 1)
('ibm oracle', 1)
('requisitos sistematizarlas', 1)
('instalar en', 1)
('institucional elaboración', 1)
('epom los', 1)
('writer compulsory', 1)
('somos un', 1)
('la circunscripción', 1)
('cambio disponibilidad', 1)
('que trámites', 1)
('evaluación establecidos', 1)
('jornadas de', 1)
('indicadores la', 1)
('buscar soluciones', 1)
('acuerdo objetivos', 1)
('negocio identificar', 1)
('equipamiento', 1)
('reporte del', 1)
('adquiridos por', 1)
('direccionar', 1)
('como políticas', 1)
('correctas', 1)
('sus próximas', 1)
('and associates', 1)
('controlados', 1)
('fiscales revisión', 1)
('externa con', 1)
('quechua nivel', 1)
('los actores', 1)
('recha

('productividad medición', 1)
('iso 9000', 1)
('saneamiento planeamiento', 1)
('accionistas apoya', 1)
('sinergias entre', 1)
('marketing consumo', 1)
('de baja', 1)
('exposición al', 1)
('límites del', 1)
('enviar el', 1)
('replica coordinación', 1)
('de julio', 1)
('como no', 1)
('basar en', 1)
('programa revisar', 1)
('con residencia', 1)
('competitividad rentabilidad', 1)
('institutions private', 1)
('establecida analizar', 1)
('preinversión nivel', 1)
('medir la', 1)
('financiamiento asegurando', 1)
('cultura humana', 1)
('estrategia efectividad', 1)
('transmitirlo', 1)
('estratégico indicadores', 1)
('establecidos alto', 1)
('determinando las', 1)
('privado enfocadas', 1)
('reforzados', 1)
('que agilicen', 1)
('salariales que', 1)
('control reducción', 1)
('cargo controlar', 1)
('dar fluidez', 1)
('optimizan los', 1)
('humanos otros', 1)
('área ejecución', 1)
('ciencias políticas', 1)
('inversion pública', 1)
('principales iniciativas', 1)
('resultados fácil', 1)
('independientes

('embarque en', 1)
('exportación con', 1)
('ilo gestión', 1)
('administrativas financiera', 1)
('importada apoyo', 1)
('trámites administrativos', 1)
('el packing', 1)
('packing list', 1)
('suministros materia', 1)
('al comercio', 1)
('100 eps', 1)
('importaciones exportaciones', 1)
('la totalidad', 1)
('almacenes de', 1)
('exterior apoyo', 1)
('llegada ingreso', 1)
('exterior sect', 1)
('los respectivos', 1)
('exterior como', 1)
('repuestos importados', 1)
('aduana agentes', 1)
('el documento', 1)
('la exportación', 1)
('factura packing', 1)
('importación costeos', 1)
('necesidades indicadas', 1)
('paita', 1)
('totalidad del', 1)
('operativas emisión', 1)
('industrial beneficios', 1)
('al agente', 1)
('aduanero al', 1)
('peligrosos en', 1)
('terrestre', 1)
('marítimos transportistas', 1)
('cliente enviada', 1)
('entrega anticipado', 1)
('coordinación entrega', 1)
('contra entrega', 1)
('aplicaciones los', 1)
('del comercio', 1)
('comercial logística', 1)
('prepara reportes', 1)
('orig

In [13]:
# Using a lemmatization dictionary
# Reading

# Post working review: 
# Lemmatization doesn't seems to work cause it reduce the number of "unique" words in low sample offers
lemmatizer = {}
with open("lemmatization-es.txt") as lemma_file:
    for line in lemma_file:
        line = line.split()                
        lemmatizer[line[0]] = line[1]


In [14]:
for label in exc:
    print("Label: ", label)
    for word in exc[label]:
        if word in lemmatizer:
            print(word, lemmatizer[word])    

Label:  MC
boletín boletines
cualquiera cualquier
redactar redactó
extracto extractos
aterrizaje aterrizajes
independiente independientes
mujer mujeres
factor factores
tutoría tutorías
entrevistar entrevistó
inmerso inmersos
francés franceses
útil útiles
sílabo sílabos
periodista periodistas
publicación publicaciones
enseñanza enseñanzas
sintetizar sintetizó
empleo empleos
domingo domingos
despegue despegues
esquematizar esquematizó
sociólogo sociólogos
disciplinario disciplinarios
actualidad actualidades
rampa rampas
guía guías
curiosidad curiosidades
método métodos
ocupación ocupaciones
esencia esencias
comentario comentarios
partida partidas
conexión conexiones
clase clases
estacionamiento estacionamientos
embajada embajadas
psicólogo psicólogos
consultar consultó
manga mangas
preferente preferentes
aerolínea aerolíneas
arma armas
migratorio migratorios
abstracción abstracciones
terminal terminales
conversar conversó
Label:  EM
noche noches
tope topes
ratio ratios
porcentual porcent

In [16]:
# Word tokenizer vs tfidf tokenizer
# Result1: Word tokenizer needs punctuation remove
from nltk import word_tokenize

punctuations = set("•!$%&*+,.//:=;ò?@]`¡¨°–“”−’´¿-()·")

def custom_tokenize(doc):
    translator = str.maketrans("".join(punctuations),' '*len(punctuations))
    doc = doc.translate(translator)    
    return [word.strip("-").replace("'", "") for word in word_tokenize(doc)]


vect1 = TfidfVectorizer(ngram_range=(1,2), max_df=0.55, norm=None, use_idf=False,
                            tokenizer=custom_tokenize)

vect2 = TfidfVectorizer(ngram_range=(1,2), max_df=0.55, norm=None, use_idf=False)

vect1.fit(X)
vect2.fit(X)

# "Words found only by word_tokenize"
only_wt = []
for word in vect1.vocabulary_:    
    if word not in vect2.vocabulary_:
        only_wt.append(word)

only_df = []
# "Words found only by default tokenize"
for word in vect2.vocabulary_:    
    if word not in vect1.vocabulary_:
         only_df.append(word)
            
print(only_wt)
        

['', 'o jefaturas', 'general 5', 'ejecución y', 'a proponer', 'f mejorar', 'm coordinación', 'vendedores y', 'financiero u', 'maxqda o', 'i archivar', 'representar a', 'y esquemas', 'categorías a', 'ayuden a', 'y hablado', 'e hacer', 'o qlickview', 'potencial y', 'y políticos', 'remisión y', 'y rolling', 'y situación', 's 5', 'decisión y', 'procesamiento y', 'y perú', 'y partida', '2012 3', 'sobreendeudamiento 4', 'invita a', 'dirección y', 'in y', 'producción o', 'interés o', 'establecer e', 'de ``', 'y flujo', '5 dominio', 'el 7', 'comprometidas y', 'y llevando', 'calle o', 'r', 'detectar y', 'cic y', 'industrias o', 'institucionales o', 'y creditos', 'y sistemas', 'y sin', '1 participar', 'minas u', 'o parte', 'profundidad a', 'ó consultoría', 'y decimo', 'y subcontratistas', 'presupuesto s', 'carreras a', 'colección y', 'las 8', '4 elaboración', 'inducciones y', 'indispensable ó', 'cuestionando y', 'réplica f', 'y préstamos', 'y fidelización', 'diseño y', 'difusión y', 'préstamos a

In [18]:
# Finding rare punctuation
common_chars = set("abcdefghijklmnopqrstuvwxyz-0123456789áéíóúñ")

rare_chars = set()
for word in only_wt:    
    for char in word:
        #if char in test_punctuations:
        if char not in common_chars:
            print(word)
            rare_chars.add(char)
            
rare_chars

o jefaturas
general 5
ejecución y
a proponer
f mejorar
m coordinación
vendedores y
financiero u
maxqda o
i archivar
representar a
y esquemas
categorías a
ayuden a
y hablado
e hacer
o qlickview
potencial y
y políticos
remisión y
y rolling
y situación
s 5
decisión y
procesamiento y
y perú
y partida
2012 3
sobreendeudamiento 4
invita a
dirección y
in y
producción o
interés o
establecer e
de ``
de ``
de ``
y flujo
5 dominio
el 7
comprometidas y
y llevando
calle o
detectar y
cic y
industrias o
institucionales o
y creditos
y sistemas
y sin
1 participar
minas u
o parte
profundidad a
ó consultoría
y decimo
y subcontratistas
presupuesto s
carreras a
colección y
las 8
4 elaboración
inducciones y
indispensable ó
cuestionando y
réplica f
y préstamos
y fidelización
diseño y
difusión y
préstamos a
académicas y
reportes ``
reportes ``
reportes ``
actualizar y
secundario y
y reducción
timing y
1 conocimientos
y tradicional
y transparencia
equipo 1
estadística 
estadística 
mercados y
6 identificar
c

tareas a
y fondos
y nuevos
proyecto y
c recuperación
s coordinar
directrices y
caja a
1 haber
s revisión
seguros o
pertenecientes a
compañía a
y arquitectura
contibuir a
essalud y
promoción e
servicios a
a operaciones
o personalmente
banca a
contribuyendo a
pmp y
y deseable
revisar y
administración o
vigilancia y
respecto a
y empaques
 at
 at
evaluadores y
y lograr
desembolsos a
y administrativos
direcciones y
mismos y
y verificaciones
o monitoreo
y encuestas
covenants y
y manejando
afectan a
área y
a negocios
operativos e
efectivas o
inmobiliarios y
estratégicos 3
h elaboración
a contabilidad
y quechua
visión y
y provincia
correcto y
y extranjeras
promoción y
y cuente
y valorados
y descripción
1 objetivos
socios y
informa a
contratos e
s contribuir
y deseas
0 de
previa y
y compromisos
y miembro
3 elaboración
campo a
y corredores
negocio 4
crediticio a
a tesorería
candidatos a
00 s
a 5
excel 5
y eeff
aspectos a
o negación
a trabajar
y estatus
individuales y
y medios
fidelización y
y 

y síntesis
y informes
stata o
financiador 7
pertinencia y
universidades o
personas a
3 tipo
o ingeniero
continua y
y verificación
e buen
comisiones f
y será
4 identificar
financieras y
captación b
y emitirlas
mejora 3
y elegibilidad
a programar
y desactivación
d actualizar
alco y
aplicadas y
tesorero a
naturales y
e ifrs
y anticiparse
aéreas o
y poner
ética y
studio y
clases y
y alta
o elaborar
promover e
interna y
supervisión a
terrestre y
ey y
técnica y
equivalentes a
y coordinando
1 coordinar
marketing o
y cómo
completa y
seguridad a
y estados
y deducciones
management y
y locales
o cancelación
s key
a incumplimiento
incorporar a
sector o
pago d
motivación e
empleados a
y estudiantes
motivación y
y operativos
pucp 2
y comercial
variación y
propuestas a
y b2c
y específicos
y financiador
3 administrar
corregir y
y hechos
y formulación
a planilla
campañas a
efectivo e
incorporen a
a finanzas
feedback a
conocimiento a
pricing a
presentación a
b contar
i misión
comunicativo y
auditoría y


{' ', '_', '`', 'º', '‐', '\uf02d', '\uf0a7', '\uf0b7', '\uf0fc', '\ufeff'}

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords


class CustomTokenizer(object):
    def __init__(self, lemmatizer):
        self.wnl = lemmatizer
        self.tok = TfidfVectorizer().build_tokenizer()
        
    def __call__(self, doc):
        return [word for word in self.tok(doc)]
    
        tokens = []
        for t in self.tok(doc):
            if t in self.wnl:
                tokens.append(self.wnl[t])
            else:
                tokens.append(t)
                
        return tokens


tfidf_vect = TfidfVectorizer(ngram_range=(1,1), max_df=0.55, norm=None, use_idf=False,
                            tokenizer=CustomTokenizer(lemmatizer))



#tfidf_vect = TfidfVectorizer(ngram_range=(1,2), max_df=0.55)
tfidf_vect.fit(X)
len(tfidf_vect.vocabulary_)

6801

In [93]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


clfs = []

clfs.append(("Ada",OneVsRestClassifier(AdaBoostClassifier(random_state=42))))
clfs.append(("Bernou", OneVsRestClassifier(BernoulliNB())))
clfs.append(("SVC ", OneVsRestClassifier(SVC())))

clfs.append(("Multi", OneVsRestClassifier(MultinomialNB())))

tf_train = tfidf_vect.transform(x_train)
tf_test = tfidf_vect.transform(x_test)
for name, clf in clfs:
    clf.fit(tf_train, y_train)    
    y_pred = clf.predict(tf_test)
    print(name, accuracy_score(y_test, y_pred))


Ada 0.406504065041
Bernou 0.406504065041
SVC  0.19512195122
Multi 0.50406504065


In [91]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1.1))
clf.fit(tf_train, y_train)
y_pred = clf.predict(tf_test)

print(accuracy_score(y_test, y_pred))

0.512195121951


In [49]:
from sklearn.grid_search import GridSearchCV
param_grid = [{'estimator__alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}]
clf = GridSearchCV(OneVsRestClassifier(MultinomialNB()), param_grid, cv=4)
tf_train = tfidf_vect.transform(x_train)
clf.fit(tf_train, y_train)
y_pred = clf.predict(tf_test)
print(name, accuracy_score(y_test, y_pred))


Multi 0.50406504065


In [369]:
pred_prob = [[format(p, '.1f') for p in probs] for probs in clf.predict_proba(tf_test)]

In [370]:
print("Classes: ", mlb.classes_)
# Economía Internacional
# Finanzas
# Teoría Económica
# Organización Industrial
# Métodos Cuantitativos/Investigación económica
# Proyectos/Planeamiento estratégico
# Estudios de mercado

for yt, yp, prob in zip(y_test, y_pred, pred_prob):
    print("True: ", yt, " Pred: ", yp , "Probs: ", prob)

Classes:  ['EI' 'EM' 'F' 'MC' 'OI' 'P' 'TE']
True:  [0 0 0 0 0 1 0]  Pred:  [0 0 1 0 0 1 0] Probs:  ['0.0', '0.0', '1.0', '0.0', '0.0', '1.0', '0.0']
True:  [0 0 1 0 0 0 0]  Pred:  [0 0 1 0 0 1 0] Probs:  ['0.0', '0.0', '1.0', '0.0', '0.0', '1.0', '0.0']
True:  [0 0 0 0 0 1 0]  Pred:  [0 0 1 0 0 1 0] Probs:  ['0.0', '0.0', '1.0', '0.0', '0.0', '1.0', '0.0']
True:  [0 0 1 0 0 1 0]  Pred:  [0 0 1 0 0 1 0] Probs:  ['0.0', '0.0', '1.0', '0.0', '0.0', '1.0', '0.0']
True:  [0 0 1 0 0 1 0]  Pred:  [0 0 1 0 0 1 0] Probs:  ['0.0', '0.0', '1.0', '0.0', '0.0', '1.0', '0.0']
True:  [0 0 0 0 0 1 0]  Pred:  [0 0 0 0 0 1 0] Probs:  ['0.0', '0.0', '0.0', '0.0', '0.0', '0.9', '0.0']
True:  [0 0 0 0 0 1 0]  Pred:  [0 0 0 0 0 1 0] Probs:  ['0.0', '0.0', '0.0', '0.0', '0.0', '1.0', '0.0']
True:  [0 0 0 0 0 1 0]  Pred:  [0 0 0 0 0 1 0] Probs:  ['0.0', '0.0', '0.0', '0.0', '0.0', '1.0', '0.0']
True:  [0 1 0 0 0 0 0]  Pred:  [0 0 1 0 0 0 0] Probs:  ['0.0', '0.0', '1.0', '0.0', '0.0', '0.3', '0.0']
True:  [0 